<a href="https://colab.research.google.com/github/gokulakrishnanbalaji/semeval-sentence-relatedness/blob/main/telugu_lang_finetuning_bard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers torch --quiet

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, util
from torch.utils.data import DataLoader
import torch
import numpy as np
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split

In [3]:
train_df=pd.read_csv('https://raw.githubusercontent.com/semantic-textual-relatedness/Semantic_Relatedness_SemEval2024/main/Track%20A/tel/tel_train.csv')
dev_df=pd.read_csv('https://raw.githubusercontent.com/semantic-textual-relatedness/Semantic_Relatedness_SemEval2024/main/Track%20A/tel/tel_dev.csv')

In [4]:
train_df[['text1', 'text2']] = train_df['Text'].str.split('\n', 1, expand=True) # Splitting into text1 and text2
train_df.drop('Text', axis=1, inplace=True) #Removing Text column, as it is redundant

<ipython-input-4-574f2c7e3864>:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  train_df[['text1', 'text2']] = train_df['Text'].str.split('\n', 1, expand=True) # Splitting into text1 and text2


In [5]:
train_df.head()

,PairID,Score,text1,text2
0,TEL-train-00001,0.88,"""జమ్మూకాశ్మీర్లో మంచుకొండచరియలు విరిగిపడి గురు...","""జమ్మూకాశ్మీర్లో మంచుకొండచరియలు విరిగిపడిన ఘటన..."
1,TEL-train-00002,0.87,"""కూతురుపైనే కన్నతండ్రి అత్యాచారానికి పాల్పడిన ...","""కన్నతండ్రి కూతురుపై అత్యాచారం చేసిన దారుణ ఘటన..."
2,TEL-train-00003,0.87,"""గంగా నదిని కాపాడేందుకు ప్రత్యేక చట్టం తీసుకుర...","""గంగానదిని రక్షించేందుకు ప్రత్యేక చట్టం తేవాలన..."
3,TEL-train-00004,0.84,"""కరోనా వైరస్పై అమెరికా ప్రభుత్వానికి వ్యతిరేకం...","""అమెరికా ప్రభుత్వానికి కరోనా వైరస్పై జరుగుతున్..."
4,TEL-train-00005,0.84,"""కరోనా వైరస్ ( కొవిడ్ - 19 ) మనుషులపైనే కాదు జ...","""కరోనావైరస్ (కోవిడ్-19) మానవులకు మరియు జంతువుల..."


In [6]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [7]:
model = SentenceTransformer('ai4bharat/indic-bert')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/345 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

spiece.vocab:   0%|          | 0.00/5.59M [00:00<?, ?B/s]

tf_model.ckpt.data-00000-of-00001:   0%|          | 0.00/400M [00:00<?, ?B/s]

tf_model.ckpt.index:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tf_model.ckpt.meta:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

In [8]:
train_examples = [
    InputExample(texts=[row["text1"], row["text2"]], label=row["Score"])
    for i, row in train_df.iterrows()
]

In [9]:
train_dataloader = DataLoader(train_examples, batch_size=8, shuffle=True)  # Adjust batch_size as needed

In [10]:
train_loss = losses.CosineSimilarityLoss(model=model)

In [11]:
def compute_spearman_correlation(predictions, targets):
    return spearmanr(predictions, targets).correlation

def on_epoch_end(epoch):
    val_scores = []  # Empty list to store all predicted scores

    for i, row in val_df.iterrows():
        text1 = row["text1"]
        text2 = row["text2"]

        # Generate individual embeddings
        embedding1 = model.encode([text1])
        embedding2 = model.encode([text2])

        # Calculate and append score
        score = util.cos_sim(embedding1, embedding2)[0][0]  # Access specific element
        val_scores.append(np.round(float(score.item()), 2))

    print(f"Spearman Coefficient on {epoch}: {spearmanr(val_scores, val_df['Score']).correlation}")

In [ ]:
num_epochs = 7
on_epoch_end(0)
for epoch in range(num_epochs):
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        warmup_steps=100,
    )
    on_epoch_end(epoch)

Spearman Coefficient on 0: 0.4068274864602115


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/117 [00:00<?, ?it/s]

Spearman Coefficient on 0: 0.5245999744559968


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/117 [00:00<?, ?it/s]

In [ ]:
dev_df.shape

In [ ]:
dev_df[['text1', 'text2']] = dev_df['Text'].str.split('\n', 1, expand=True) # Splitting into text1 and text2
dev_df.drop('Text', axis=1, inplace=True) #Removing Text column, as it is redundant

In [ ]:
all_scores = []  # Empty list to store all predicted scores

for i, row in dev_df.iterrows():
    text1 = row["text1"]
    text2 = row["text2"]

    # Generate individual embeddings
    embedding1 = model.encode([text1])
    embedding2 = model.encode([text2])

    # Calculate and append score
    score = util.cos_sim(embedding1, embedding2)[0][0]  # Access specific element
    all_scores.append(np.round(float(score.item()), 2))

In [ ]:
dev_df["Pred_Score"] = np.round(all_scores, 2)

In [ ]:
dev_df = dev_df[['PairID','Pred_Score']]

In [ ]:
dev_df

In [ ]:
dev_df.to_csv('pred_tel_a.csv', index=False)